<a href="https://colab.research.google.com/github/IgorNascAlves/Validacao-de-chaves-aleatorias-FIPS-140-1/blob/master/Valida%C3%A7%C3%A3o_de_chaves_aleat%C3%B3rias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import Counter
from BitVector import BitVector

In [2]:
data = pd.read_csv('Matrix2.txt', sep="'", header=None)
data = data.drop(columns=[0, 2])
data = pd.DataFrame([f"{BitVector(hexstring = item)}" for item in data[1]])

In [9]:
contador_poker = 1
contador_monobit = 1
contador_long_run_test = 1
contador_long_test = 1

## THE MONOBIT TEST

In [10]:
def Monobit(chave):
    
    global contador_monobit
    contador = []
    i = 0
    
    c = Counter(chave)
    contador.append(c['1'])
        
    for item in contador:
        if item > 9654 and item < 10346:
            i = i + 1
    if(i == 0):
        print('A chave [{}] não passou no The MonoBit Test'.format(contador_monobit))
        contador_monobit = contador_monobit +1
    else:
        print('A chave [{}] passou no The MonoBit Test'.format(contador_monobit))
        contador_monobit = contador_monobit +1

### THE POKER TEST

In [11]:
def PokerTest(chave):
    
    global contador_poker
    contador_por_chave = []
    contador = []
    nibble = ''
    nibble_por_chave = {}          
    total_nibble_por_chave = []
    i = 0
    
    for valor in chave:
        if i == 4:
            contador.append(nibble)
            i = 0
            nibble = ''
        nibble = nibble + valor
        i = i + 1
    contador_por_chave.append(contador)
    contador = []
    contador_por_chave = pd.DataFrame(contador_por_chave)
    
    #Count and store the number of occurrences of each of the 16 possible 4 bit values.
    nibbles = ['0000','0001','0010','0011','0100','0101','0110','0111','1000','1001','1010','1011','1100','1101','1110','1111',]
    
    
    for chave in contador_por_chave.values:           
        for nibble in nibbles:
            c = Counter(chave)
            nibble_por_chave[nibble] = c[nibble]
        total_nibble_por_chave.append(nibble_por_chave)
        nibble_por_chave = {}
    total_nibble_por_chave = pd.DataFrame(total_nibble_por_chave)
    
    #Denote f(i) as the number of each 4 bit value i where 0 s is 15.
    soma_chaves = []
    acumulado = 0
    x = 0
    #chave = total_nibble_por_chave[:1]
    for chave in total_nibble_por_chave.values:
        for valor in chave:
            acumulado = acumulado + valor * valor
        x = (16/5000) * acumulado - 5000
        soma_chaves.append(x)
        x = 0
        acumulado = 0
    soma_chaves = pd.DataFrame(soma_chaves)

 
    #The test is passed if 1.03 < X < 57.4.
    i = 0
    for chave in soma_chaves.values:
        if chave > 1.03 and chave < 57.4:
            i = i + 1
    
    if(i == 0):
        print('A chave [{}] não passou no The Poker Test'.format(contador_poker))
        contador_poker = contador_poker +1
    else:
        print('A chave [{}] passou no The Poker Test'.format(contador_poker))
        contador_poker = contador_poker +1

## THE LONG RUN TEST

In [12]:
def LongRunTest(chave):
    
    global contador_long_run_test
    contador_por_chave = []
    contador = []
    salvo = ''
    i = 0

    for item in chave:
        if salvo == '':
            salvo = item
        else:  
            if item == salvo:
                i = i + 1
            else:
                contador.append(i)
                i = 0
                salvo = item
                
    contador_por_chave.append(contador)
    contador = []
    salvo = ''
    contador_por_chave = pd.DataFrame(contador_por_chave)

    i = 0
    for contador in contador_por_chave.values:
        if len(contador[contador > 33]) == 0:
            i = i + 1
   
    if(i == 0):
        print('A chave [{}] não passou no The Long Run Test'.format(contador_long_run_test))
        contador_long_run_test = contador_long_run_test +1
    else:
        print('A chave [{}] passou no The Long Run Test'.format(contador_long_run_test))
        contador_long_run_test = contador_long_run_test +1         

## THE RUN TEST

In [13]:
def RunTest(chave):
    
    global contador_long_test
    contador_label = {'1': 0,'2': 0,'3': 0,'4': 0,'5': 0,'6': 0,}
    contador_por_chave = []
    salvo = ''
    i = 0
  
    for item in chave:
        if salvo == '':
            salvo = item
            i = i + 1
        else:  
            if item == salvo:
                i = i + 1
            else:
                if i >= 6:
                    contador_label['6'] = contador_label['6'] + 1
                if i == 5:
                    contador_label['5'] = contador_label['5'] + 1
                if i == 4:
                    contador_label['4'] = contador_label['4'] + 1
                if i == 3:
                    contador_label['3'] = contador_label['3'] + 1
                if i == 2:
                    contador_label['2'] = contador_label['2'] + 1
                if i == 1:
                    contador_label['1'] = contador_label['1'] + 1
                i = 0
                salvo = item

    contador_por_chave.append(contador_label)
    salvo = ''
    contador_por_chave = pd.DataFrame(contador_por_chave)

    cont = 0

    for contador in contador_por_chave.values:
        if contador[5] < 90 or contador[5] > 223:
            cont = cont + 1
        elif contador[4] < 90 or contador[4] > 223:
            cont = cont + 1
        elif contador[3] < 223 or contador[3] > 402:
            cont = cont + 1
        elif contador[2] < 502 or contador[2] > 748:
            cont = cont + 1
        elif contador[1] < 1079 or contador[1] > 1421:
            cont = cont + 1
        elif contador[0] < 2267 or contador[0] > 2733:
            cont = cont + 1
    
    if(i == 0):
        print('A chave [{}] não passou no The Run Test'.format(contador_long_test))
        contador_long_test = contador_long_test +1
    else:
        print('A chave [{}] passou no The Run Test'.format(contador_long_test))
        contador_long_test = contador_long_test +1     

# MAIN

In [14]:
def main():
    for i in range(len(data)):
        Monobit(data[0][i])
        PokerTest(data[0][i])
        LongRunTest(data[0][i])
        RunTest(data[0][i])
        print('\n')

In [15]:
main()

A chave [1] não passou no The MonoBit Test
A chave [1] não passou no The Poker Test
A chave [1] passou no The Long Run Test
A chave [1] não passou no The Run Test


A chave [2] não passou no The MonoBit Test
A chave [2] não passou no The Poker Test
A chave [2] passou no The Long Run Test
A chave [2] passou no The Run Test


A chave [3] não passou no The MonoBit Test
A chave [3] não passou no The Poker Test
A chave [3] passou no The Long Run Test
A chave [3] passou no The Run Test


A chave [4] não passou no The MonoBit Test
A chave [4] não passou no The Poker Test
A chave [4] passou no The Long Run Test
A chave [4] não passou no The Run Test


A chave [5] não passou no The MonoBit Test
A chave [5] não passou no The Poker Test
A chave [5] passou no The Long Run Test
A chave [5] não passou no The Run Test


A chave [6] não passou no The MonoBit Test
A chave [6] não passou no The Poker Test
A chave [6] passou no The Long Run Test
A chave [6] não passou no The Run Test


A chave [7] não pa